In [1]:
import pandas as pd
import numpy as np
import json
import joblib
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

from src.data.tokenizers.basic_tokenizer import BasicWordTokenizer, CustomWordCharTokenizer
from src.data.datastruct import Batch, Sample
from src.data.collate import create_batch, create_samples
from src.models.neural_text_classifier import TicketTextClassifierV01
from src.training.train_neural import train_model, train_model_cuda
from src.evaluation.metrics import format_cm, evaluate
from src.evaluation.neural_eval import evaluate as neural_eval, evaluate_cuda as neural_eval_cuda

#### similarity index

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
with open("../data/processed/agumented_ticketdata_similarity_v01.json", "r") as f:
    agumented_data = json.load(f)

X_train_agumented = agumented_data["train"]["X"]
X_test_agumented = agumented_data["test"]["X"]

y_train = agumented_data["train"]["y"]
y_test = agumented_data["test"]["y"]

labelencode: LabelEncoder = joblib.load("../artifacts/labelencoder_neural_v01.pkl")

y_train_encoded = labelencode.transform(y_train)
y_test_encoded = labelencode.transform(y_test)

criterion = nn.CrossEntropyLoss()

tokenizer_uni = BasicWordTokenizer()
tokenizer_uni_char3_5 = CustomWordCharTokenizer()

tokenizer_uni.fit(X_train_agumented)
tokenizer_uni_char3_5.fit(X_train_agumented)

train_samples_uni = create_samples(X_train_agumented, y_train_encoded, tokenizer_uni)
test_samples_uni = create_samples(X_test_agumented, y_test_encoded, tokenizer_uni)

train_samples_uni_char3_5 = create_samples(X_train_agumented, y_train_encoded, tokenizer_uni_char3_5)
test_samples_uni_char3_5 = create_samples(X_test_agumented, y_test_encoded, tokenizer_uni_char3_5)


EPOCHS = 50
N_CLASS = 8 # known from previous notebooks
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 64
EMBEDDING_DIM = 256

Tokenizer fitted. Vocab size:  11608
Tokenizer fitted. Vocab size:  30002


In [4]:
model_uni = TicketTextClassifierV01(
    len(tokenizer_uni), EMBEDDING_DIM, pad_id=tokenizer_uni.get_pad_id(), n_classes=N_CLASS
).to(device)

In [5]:
test_batches_uni = create_batch(
    TEST_BATCH_SIZE, in_samples=test_samples_uni, pad_id=tokenizer_uni.get_pad_id(), device=device,
    shuffle=False, random_state=43
)

In [6]:
history_uni = train_model(
    model=model_uni, total_epochs=EPOCHS, train_samples=train_samples_uni, training_batch_size=TRAIN_BATCH_SIZE,
    pad_id=tokenizer_uni.get_pad_id(), device=device, test_batches=test_batches_uni, criterion=criterion, early_stop=True,
    random_state=43
)

  2%|▏         | 1/50 [00:11<09:20, 11.43s/it]


        Epoch 1
        Train Loss: 1.6904 | Train F1: 0.2654
        Val   Loss: 1.5601 | Val   F1: 0.3829
        


  4%|▍         | 2/50 [00:22<08:44, 10.93s/it]


        Epoch 2
        Train Loss: 1.4780 | Train F1: 0.4355
        Val   Loss: 1.5049 | Val   F1: 0.4439
        


  6%|▌         | 3/50 [00:32<08:30, 10.86s/it]

...


        Epoch 20
        Train Loss: 1.2736 | Train F1: 0.5724
        Val   Loss: 1.4503 | Val   F1: 0.5210
        


 42%|████▏     | 21/50 [03:30<04:40,  9.67s/it]


        Epoch 21
        Train Loss: 1.2684 | Train F1: 0.5732
        Val   Loss: 1.4513 | Val   F1: 0.5210
        


 42%|████▏     | 21/50 [03:40<05:04, 10.50s/it]


        Epoch 22
        Train Loss: 1.2646 | Train F1: 0.5726
        Val   Loss: 1.4521 | Val   F1: 0.5207
        
Early stopping at epoch: 22


In [19]:
loss_uni, y_pred_uni, y_true_uni = neural_eval(
    model=model_uni, batches=test_batches_uni, criterion=criterion
)
format_cm(evaluate(y_true_uni, y_pred_uni), class_names=list(labelencode.classes_), normalize=True)

              precision    recall  f1-score   support

           0     0.5553    0.5284    0.5415      1425
           1     0.5252    0.3551    0.4237       352
           2     0.5581    0.5525    0.5552      2183
           3     0.5256    0.6358    0.5755      2724
           4     0.5651    0.4198    0.4817       424
           5     0.5004    0.4483    0.4729      1412
           6     0.6159    0.5659    0.5899       493
           7     0.5216    0.4559    0.4865       555

    accuracy                         0.5392      9568
   macro avg     0.5459    0.4952    0.5159      9568
weighted avg     0.5399    0.5392    0.5365      9568



,Pred: Access,Pred: Administrative rights,Pred: HR Support,Pred: Hardware,Pred: Internal Project,Pred: Miscellaneous,Pred: Purchase,Pred: Storage
True: Access,0.528421,0.011228,0.118596,0.210526,0.016140,0.076491,0.017544,0.021053
True: Administrative rights,0.065341,0.355114,0.113636,0.332386,0.005682,0.068182,0.028409,0.031250
True: HR Support,0.076042,0.005955,0.552451,0.215758,0.011910,0.083372,0.018781,0.035731
True: Hardware,0.080029,0.013216,0.132893,0.635830,0.015051,0.082599,0.017621,0.022761
True: Internal Project,0.075472,0.014151,0.158019,0.212264,0.419811,0.087264,0.011792,0.021226
True: Miscellaneous,0.070113,0.014873,0.143768,0.265581,0.015581,0.448300,0.021955,0.019830
True: Purchase,0.046653,0.020284,0.087221,0.188641,0.016227,0.046653,0.565923,0.028398
True: Storage,0.075676,0.019820,0.127928,0.210811,0.027027,0.057658,0.025225,0.455856


In [4]:
EPOCHS = 50
N_CLASS = 8 # known from previous notebooks
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 64
EMBEDDING_DIM = 256

In [5]:
model_uni_char3_5 = TicketTextClassifierV01(
    len(tokenizer_uni_char3_5), EMBEDDING_DIM, pad_id=tokenizer_uni_char3_5.get_pad_id(), n_classes=N_CLASS
).to(device)

In [6]:
test_batches_uni_char3_5 = create_batch(
    TEST_BATCH_SIZE, in_samples=test_samples_uni_char3_5, pad_id=tokenizer_uni_char3_5.get_pad_id(), device="cpu",
    shuffle=False, random_state=43
)

In [7]:
history_uni_char3_5 = train_model_cuda(
    model=model_uni_char3_5, total_epochs=EPOCHS, train_samples=train_samples_uni_char3_5, training_batch_size=TRAIN_BATCH_SIZE,
    pad_id=tokenizer_uni_char3_5.get_pad_id(), test_batches=test_batches_uni_char3_5, criterion=criterion, early_stop=True,
    random_state=43
)

  2%|▏         | 1/50 [01:06<54:21, 66.56s/it]


        Epoch 1
        Train Loss: 1.6674 | Train F1: 0.2660
        Val   Loss: 1.5356 | Val   F1: 0.3864
        


  4%|▍         | 2/50 [02:03<48:52, 61.09s/it]


        Epoch 2
        Train Loss: 1.4501 | Train F1: 0.4520
        Val   Loss: 1.4823 | Val   F1: 0.4510
        


  6%|▌         | 3/50 [03:01<46:41, 59.61s/it]

...


        Epoch 21
        Train Loss: 1.2568 | Train F1: 0.5756
        Val   Loss: 1.4498 | Val   F1: 0.5203
        


 44%|████▍     | 22/50 [21:47<29:13, 62.64s/it]


        Epoch 22
        Train Loss: 1.2519 | Train F1: 0.5760
        Val   Loss: 1.4512 | Val   F1: 0.5202
        


 44%|████▍     | 22/50 [22:51<29:05, 62.32s/it]


        Epoch 23
        Train Loss: 1.2476 | Train F1: 0.5775
        Val   Loss: 1.4530 | Val   F1: 0.5197
        
Early stopping at epoch: 23


In [8]:
loss_uni_char3_5, y_pred_uni_char3_5, y_true_uni_char3_5 = neural_eval_cuda(
    model=model_uni_char3_5, batches=test_batches_uni_char3_5, criterion=criterion
)
format_cm(evaluate(y_true_uni_char3_5, y_pred_uni_char3_5), class_names=list(labelencode.classes_), normalize=True)

              precision    recall  f1-score   support

           0     0.5509    0.5354    0.5431      1425
           1     0.4965    0.3977    0.4416       352
           2     0.5534    0.5552    0.5543      2183
           3     0.5434    0.6156    0.5773      2724
           4     0.5353    0.4292    0.4764       424
           5     0.4950    0.4561    0.4748      1412
           6     0.6137    0.5801    0.5965       493
           7     0.5116    0.4775    0.4939       555

    accuracy                         0.5402      9568
   macro avg     0.5375    0.5059    0.5197      9568
weighted avg     0.5394    0.5402    0.5385      9568



,Pred: Access,Pred: Administrative rights,Pred: HR Support,Pred: Hardware,Pred: Internal Project,Pred: Miscellaneous,Pred: Purchase,Pred: Storage
True: Access,0.535439,0.016842,0.127719,0.183158,0.016842,0.079298,0.017544,0.023158
True: Administrative rights,0.062500,0.397727,0.119318,0.286932,0.005682,0.073864,0.028409,0.025568
True: HR Support,0.074668,0.008704,0.555199,0.201099,0.014659,0.087952,0.019240,0.038479
True: Hardware,0.084802,0.016153,0.136197,0.615639,0.019090,0.084068,0.019090,0.024963
True: Internal Project,0.087264,0.016509,0.153302,0.183962,0.429245,0.094340,0.014151,0.021226
True: Miscellaneous,0.072946,0.017705,0.141643,0.249292,0.016997,0.456091,0.021955,0.023371
True: Purchase,0.050710,0.024341,0.091278,0.150101,0.018256,0.050710,0.580122,0.034483
True: Storage,0.073874,0.019820,0.131532,0.187387,0.027027,0.057658,0.025225,0.477477
